# scraper抓取器详解
　　参考：[scrapy启动流程源码分析(5)Scraper刮取器](https://blog.csdn.net/csdn_yym/article/details/85577613)<br>
　　　　　[scrapy源码3：scraper的源码分析](https://cuiyonghua.blog.csdn.net/article/details/108315506)<br>
     
## 一、简介
　　本节对ExecutionEngine执行引擎篇出现的Scraper进行展开。<br>
　　scraper模块是实现爬虫组件去解析响应流并且提取数据的。具体讲就是对spider中间件进行管理，通过中间件完成请求、响应、数据分析等工作。 
## 二、Scraper对象
　　Scraper对象，主要是通过3个对象：spidermw，itemproc，concurrent_items 完成请求处理、响应处理、数据处理工作的。<br>
  

In [ ]:
#  scrapy/core/scraper.py
class Scraper(object):
    def __init__(self, crawler):
        self.slot = None
        self.spidermw = SpiderMiddlewareManager.from_crawler(crawler)
        itemproc_cls = load_object(crawler.settings['ITEM_PROCESSOR'])
        self.itemproc = itemproc_cls.from_crawler(crawler)
        self.concurrent_items = crawler.settings.getint('CONCURRENT_ITEMS')
        self.crawler = crawler
        self.signals = crawler.signals
        self.logformatter = crawler.logformatter


### 2.1、spidermw对象
　　SpiderMiddlewareManger爬虫中间件管理器。<br>
　　self.spidermw = SpiderMiddlewareManager.from_crawler(crawler)，同样通过from_crawler方法生成爬虫中间件管理器。
这个from_cralwer方法是基类MiddlewareManger（scrapy/middleware.py ->MiddlewareManager）的方法。<br>
　　1、首先调用_get_mwlist_from_settings方法从配置文件中生成中间件列表。<br>
　　2、然后依次加载中间件模块并构造对象，构造顺序是先尝试调用from_cralwer,再尝试调用from_settings,最后都没有再调用init（从这里可以看出中间件的加载逻辑，from_cralwer优先于init，中间件直接在配置文件中注册名称使用，不必继承基类）。<br>
　　3、中间件除了类路径，还有一个优先级，这个决定了后面调用的先后顺序，数字越小调用越靠前。<br>
　　默认中间件：<br>
>SPIDER_MIDDLEWARES_BASE = {<br>
　　\# Engine side<br>
　　'scrapy.spidermiddlewares.httperror.HttpErrorMiddleware': 50,<br>
　　'scrapy.spidermiddlewares.offsite.OffsiteMiddleware': 500,<br>
　　'scrapy.spidermiddlewares.referer.RefererMiddleware': 700,<br>
　　'scrapy.spidermiddlewares.urllength.UrlLengthMiddleware': 800,<br>
　　'scrapy.spidermiddlewares.depth.DepthMiddleware': 900,<br>
　　\# Spider side<br>
}<br>

　　4、spider中间件默认能够识别的所有信号相关处理函数为：open_spider，close_spider，process_spider_input，process_spider_output，process_spider_exception，process_start_requests。<br>



In [ ]:
# scrapy/core/spidermw.py
"""
Spider Middleware manager

See documentation in docs/topics/spider-middleware.rst
"""
from itertools import chain, islice

import six
from twisted.python.failure import Failure
from scrapy.exceptions import _InvalidOutput
from scrapy.middleware import MiddlewareManager
from scrapy.utils.defer import mustbe_deferred
from scrapy.utils.conf import build_component_list
from scrapy.utils.python import MutableChain


def _isiterable(possible_iterator):
    return hasattr(possible_iterator, '__iter__')


class SpiderMiddlewareManager(MiddlewareManager):

    component_name = 'spider middleware'

    @classmethod
    def _get_mwlist_from_settings(cls, settings):
        return build_component_list(settings.getwithbase('SPIDER_MIDDLEWARES'))

    def _add_middleware(self, mw):
        super(SpiderMiddlewareManager, self)._add_middleware(mw)
        if hasattr(mw, 'process_spider_input'):
            self.methods['process_spider_input'].append(mw.process_spider_input)
        if hasattr(mw, 'process_start_requests'):
            self.methods['process_start_requests'].appendleft(mw.process_start_requests)
        self.methods['process_spider_output'].appendleft(getattr(mw, 'process_spider_output', None))
        self.methods['process_spider_exception'].appendleft(getattr(mw, 'process_spider_exception', None))

    def scrape_response(self, scrape_func, response, request, spider):
        fname = lambda f:'%s.%s' % (
                six.get_method_self(f).__class__.__name__,
                six.get_method_function(f).__name__)

        def process_spider_input(response):
            for method in self.methods['process_spider_input']:
                try:
                    result = method(response=response, spider=spider)
                    if result is not None:
                        raise _InvalidOutput('Middleware {} must return None or raise an exception, got {}' \
                                             .format(fname(method), type(result)))
                except _InvalidOutput:
                    raise
                except Exception:
                    return scrape_func(Failure(), request, spider)
            return scrape_func(response, request, spider)

        def process_spider_exception(_failure, start_index=0):
            exception = _failure.value
            # don't handle _InvalidOutput exception
            if isinstance(exception, _InvalidOutput):
                return _failure
            method_list = islice(self.methods['process_spider_exception'], start_index, None)
            for method_index, method in enumerate(method_list, start=start_index):
                if method is None:
                    continue
                result = method(response=response, exception=exception, spider=spider)
                if _isiterable(result):
                    # stop exception handling by handing control over to the
                    # process_spider_output chain if an iterable has been returned
                    return process_spider_output(result, method_index+1)
                elif result is None:
                    continue
                else:
                    raise _InvalidOutput('Middleware {} must return None or an iterable, got {}' \
                                         .format(fname(method), type(result)))
            return _failure

        def process_spider_output(result, start_index=0):
            # items in this iterable do not need to go through the process_spider_output
            # chain, they went through it already from the process_spider_exception method
            recovered = MutableChain()

            def evaluate_iterable(iterable, index):
                try:
                    for r in iterable:
                        yield r
                except Exception as ex:
                    exception_result = process_spider_exception(Failure(ex), index+1)
                    if isinstance(exception_result, Failure):
                        raise
                    recovered.extend(exception_result)

            method_list = islice(self.methods['process_spider_output'], start_index, None)
            for method_index, method in enumerate(method_list, start=start_index):
                if method is None:
                    continue
                # the following might fail directly if the output value is not a generator
                try:
                    result = method(response=response, result=result, spider=spider)
                except Exception as ex:
                    exception_result = process_spider_exception(Failure(ex), method_index+1)
                    if isinstance(exception_result, Failure):
                        raise
                    return exception_result
                if _isiterable(result):
                    result = evaluate_iterable(result, method_index)
                else:
                    raise _InvalidOutput('Middleware {} must return an iterable, got {}' \
                                         .format(fname(method), type(result)))

            return chain(result, recovered)

        dfd = mustbe_deferred(process_spider_input, response)
        dfd.addCallbacks(callback=process_spider_output, errback=process_spider_exception)
        return dfd

    def process_start_requests(self, start_requests, spider):
        return self._process_chain('process_start_requests', start_requests, spider)


In [ ]:
# scrapy/middleware.py
from collections import defaultdict, deque
import logging
import pprint

from scrapy.exceptions import NotConfigured
from scrapy.utils.misc import create_instance, load_object
from scrapy.utils.defer import process_parallel, process_chain, process_chain_both

logger = logging.getLogger(__name__)


class MiddlewareManager(object):
    """Base class for implementing middleware managers"""

    component_name = 'foo middleware'

    def __init__(self, *middlewares):
        self.middlewares = middlewares
        self.methods = defaultdict(deque)
        for mw in middlewares:
            self._add_middleware(mw)

    @classmethod
    def _get_mwlist_from_settings(cls, settings):
        raise NotImplementedError

    @classmethod
    def from_settings(cls, settings, crawler=None):
        mwlist = cls._get_mwlist_from_settings(settings)   # 调用_get_mwlist_from_settings方法从配置文件中生成中间件列表。
        middlewares = []
        enabled = []
        for clspath in mwlist:
            try:
                mwcls = load_object(clspath)
                mw = create_instance(mwcls, settings, crawler)
                middlewares.append(mw)
                enabled.append(clspath)
            except NotConfigured as e:
                if e.args:
                    clsname = clspath.split('.')[-1]
                    logger.warning("Disabled %(clsname)s: %(eargs)s",
                                   {'clsname': clsname, 'eargs': e.args[0]},
                                   extra={'crawler': crawler})

        logger.info("Enabled %(componentname)ss:\n%(enabledlist)s",
                    {'componentname': cls.component_name,
                     'enabledlist': pprint.pformat(enabled)},
                    extra={'crawler': crawler})
        return cls(*middlewares)

    @classmethod
    def from_crawler(cls, crawler):
        return cls.from_settings(crawler.settings, crawler)

    def _add_middleware(self, mw):
        if hasattr(mw, 'open_spider'):
            self.methods['open_spider'].append(mw.open_spider)
        if hasattr(mw, 'close_spider'):
            self.methods['close_spider'].appendleft(mw.close_spider)

    def _process_parallel(self, methodname, obj, *args):
        return process_parallel(self.methods[methodname], obj, *args)

    def _process_chain(self, methodname, obj, *args):
        return process_chain(self.methods[methodname], obj, *args)

    def _process_chain_both(self, cb_methodname, eb_methodname, obj, *args):
        return process_chain_both(self.methods[cb_methodname], \
            self.methods[eb_methodname], obj, *args)

    def open_spider(self, spider):
        return self._process_parallel('open_spider', spider)

    def close_spider(self, spider):
        return self._process_parallel('close_spider', spider)


#  scrapy/utils/misc.py  -> create_instance    
def create_instance(objcls, settings, crawler, *args, **kwargs):
    """Construct a class instance using its ``from_crawler`` or
    ``from_settings`` constructors, if available.

    At least one of ``settings`` and ``crawler`` needs to be different from
    ``None``. If ``settings `` is ``None``, ``crawler.settings`` will be used.
    If ``crawler`` is ``None``, only the ``from_settings`` constructor will be
    tried.

    ``*args`` and ``**kwargs`` are forwarded to the constructors.

    Raises ``ValueError`` if both ``settings`` and ``crawler`` are ``None``.
    """
    if settings is None:
        if crawler is None:
            raise ValueError("Specifiy at least one of settings and crawler.")
        settings = crawler.settings
    if crawler and hasattr(objcls, 'from_crawler'):
        return objcls.from_crawler(crawler, *args, **kwargs)
    elif hasattr(objcls, 'from_settings'):
        return objcls.from_settings(settings, *args, **kwargs)
    else:
        return objcls(*args, **kwargs)

### 2.2、itemproc对象
　　itemproc：ItemPipelineManager项目管道管理器。<br>
>itemproc_cls = load_object(crawler.settings[‘ITEM_PROCESSOR’])<br>
self.itemproc = itemproc_cls.from_crawler(crawler)<br>

　　itemproc从配置文件中获取‘ITEM_PROCESSOR’，默认为：<br>
>ITEM_PROCESSOR = ‘scrapy.pipelines.ItemPipelineManager’<br>

　　同样通过from_crawler方法生成项目管道管理器，且ItemPipelineManager同样继承于MiddlewareManager，也属于中间件。<br>
　　默认项目管道中间件为空，ITEM_PIPELINES_BASE = {}。<br>
　　项目管道能够识别的信号处理函数：open_spider，close_spider，process_item。<br>


In [ ]:
# scrapy/pipelines/init.py
"""
Item pipeline

See documentation in docs/item-pipeline.rst
"""

from scrapy.middleware import MiddlewareManager
from scrapy.utils.conf import build_component_list

class ItemPipelineManager(MiddlewareManager):

    component_name = 'item pipeline'

    @classmethod
    def _get_mwlist_from_settings(cls, settings):
        return build_component_list(settings.getwithbase('ITEM_PIPELINES'))

    def _add_middleware(self, pipe):
        super(ItemPipelineManager, self)._add_middleware(pipe)
        if hasattr(pipe, 'process_item'):
            self.methods['process_item'].append(pipe.process_item)

    def process_item(self, item, spider):
        return self._process_chain('process_item', item, spider)


### 2.3、concurrent_items 数据并发处理控制
>self.concurrent_items = crawler.settings.getint(‘CONCURRENT_ITEMS’)，默认配置值为100。

　　这个并发数用于控制同时处理的抓取到的 item 数目，通过twisted.internet的task.Cooperator实现并发控制。
　　scraper在处理spider的parse结果后会调用handle_spider_output来处理输出，通过parallel来控制同时处理的条目。
       


In [ ]:
# scrapy/core/scraper.py 

    def handle_spider_output(self, result, request, response, spider):
        if not result:
            return defer_succeed(None)
        it = iter_errback(result, self.handle_spider_error, request, response, spider)
        dfd = parallel(it, self.concurrent_items,
            self._process_spidermw_output, request, response, spider)
        return dfd

In [ ]:
# scrapy.core.scraper.py
import logging
from collections import deque

from twisted.python.failure import Failure
from twisted.internet import defer      # 导入Failure和defer延迟。

from scrapy.utils.defer import defer_result, defer_succeed, parallel, iter_errback
from scrapy.utils.spider import iterate_spider_output
from scrapy.utils.misc import load_object   # 将导入str转为对象的。
from scrapy.utils.log import logformatter_adapter, failure_to_exc_info
from scrapy.exceptions import CloseSpider, DropItem, IgnoreRequest   # item被丢弃的异常、关闭爬虫的异常、忽略请求的异常。
from scrapy import signals
from scrapy.http import Request, Response
from scrapy.item import BaseItem
from scrapy.core.spidermw import SpiderMiddlewareManager
from scrapy.utils.request import referer_str

logger = logging.getLogger(__name__)


# 下面先分析一下导入的方法意义。
def defer_result(result):
    """
    对result进行判定， 如果是Deferred对象，那就返回 result;
    如果是Failure,返回defer_fail(result);
    其他的情况,返回defer_succeed(result)
    """
    if isinstance(result, defer.Deferred):
        return result
    elif isinstance(result, failure.Failure):
        return defer_fail(result)
    else:
        return defer_succeed(result)
    
# 再看看defer_fail，defer_succeed 都做了什么 
def defer_fail(_failure):
    """Same as twisted.internet.defer.fail but delay calling errback until
    next reactor loop
    It delays by 100ms so reactor has a chance to go trough readers and writers
    before attending pending delayed calls, so do not set delay to zero.
    与twisted.internet.defer.fail相同，但将调用errback延迟到下一个reactor循环
    它延迟100ms，因此reactor在处理等待的调用前，有时间完成读写操作，所以不要设置延迟为零。
    """
    d = defer.Deferred()
    reactor.callLater(0.1, d.errback, _failure)
    return d

def defer_succeed(result):
    """Same as twisted.internet.defer.succeed but delay calling callback until
    next reactor loop

    It delays by 100ms so reactor has a chance to go trough readers and writers
    before attending pending delayed calls, so do not set delay to zero.
    同上面defer_fail,延迟100ms,以完成读写操作
    """
    d = defer.Deferred()
    reactor.callLater(0.1, d.callback, result)
    return d

def parallel(iterable, count, callable, *args, **named):
    """Execute a callable over the objects in the given iterable, in parallel,
    using no more than ``count`` concurrent calls.
    并行地对给定iterable中的对象执行一个可调用对象，使用的并发数不超过 count 。
    Taken from: http://jcalderone.livejournal.com/24285.html
    """
    coop = task.Cooperator()
    work = (callable(elem, *args, **named) for elem in iterable)
    return defer.DeferredList([coop.coiterate(work) for _ in range(count)])

def iter_errback(iterable, errback, *a, **kw):
    """Wraps an iterable calling an errback if an error is caught while
    iterating it.
    迭代处理iterable这个对象，如果在迭代时捕获到错误，则可以包装一个可调用的回调函数。
    """
    it = iter(iterable)
    while True:
        try:
            yield next(it)
        except StopIteration:
            break
        except:
            errback(failure.Failure(), *a, **kw)

def iterate_spider_output(result):
    # 这个方法定位到另一个位置了。 我们追踪过去看看
    return arg_to_iter(result)

def arg_to_iter(arg):
    """Convert an argument to an iterable. The argument can be a None, single
    value, or an iterable.
    将对象转化为可迭代的对象。如果是none返回[], 如果是不可迭代就返回[arg] 本身可迭代的就返回自身。
    _ITERABLE_SINGLE_VALUES = dict, BaseItem, six.text_type, bytes
    Exception: if arg is a dict, [arg] will be returned
    """
    if arg is None:
        return []
    elif not isinstance(arg, _ITERABLE_SINGLE_VALUES) and hasattr(arg, '__iter__'):
        return arg
    else:
        return [arg]   
 
def logformatter_adapter(logkws):
    """
    Helper that takes the dictionary output from the methods in LogFormatter
    and adapts it into a tuple of positional arguments for logger.log calls,
    handling backward compatibility as well.
    # 这个方法是个适配器，将logkws字典对象最终返回一个元组，日志级别，信息，和详细参数。
    """
    if not {'level', 'msg', 'args'} <= set(logkws):
        warnings.warn('Missing keys in LogFormatter method',
                      ScrapyDeprecationWarning)

    if 'format' in logkws:
        warnings.warn('`format` key in LogFormatter methods has been '
                      'deprecated, use `msg` instead',
                      ScrapyDeprecationWarning)

    level = logkws.get('level', logging.INFO)
    message = logkws.get('format', logkws.get('msg'))
    # NOTE: This also handles 'args' being an empty dict, that case doesn't
    # play well in logger.log calls
    args = logkws if not logkws.get('args') else logkws['args']
    return (level, message, args)

def failure_to_exc_info(failure):
    """Extract exc_info from Failure instances"""
    # 用于提取错误实例的信息。
    if isinstance(failure, Failure):
        return (failure.type, failure.value, failure.getTracebackObject())


def referer_str(request):
    """ Return Referer HTTP header suitable for logging. """
    # 从请求头信息里面获取referer信息。
    referrer = request.headers.get('Referer')
    if referrer is None:
        return referrer
    return to_native_str(referrer, errors='replace')


class Slot(object):
    """Scraper slot (one per running spider)"""

    MIN_RESPONSE_SIZE = 1024
    
    def __init__(self, max_active_size=5000000):
        self.max_active_size = max_active_size
        self.queue = deque()
        self.active = set()
        self.active_size = 0
        self.itemproc_size = 0
        self.closing = None
        # 这里设置了，最大活动大小，默认值为5000000， 这个值为何不放到默认配置文件里面呢。疑惑下。
        # 构造一个deques,使用集合去存储活动的，活动的大小开始为0
        # itemproc_size item处理的大小，关闭状态为none.
        
    def add_response_request(self, response, request):
        deferred = defer.Deferred()
        self.queue.append((response, request, deferred))
        if isinstance(response, Response):
            self.active_size += max(len(response.body), self.MIN_RESPONSE_SIZE)
        else:
            self.active_size += self.MIN_RESPONSE_SIZE
        return deferred
        # 这个方法从名字上看 ，应该是添加响应请求吧，
        # 创建一个defer对象，队列里面添加一个(response,request,deferred)元祖
        # 如果response 是Resposne子类的话，活动的大小就是原来活动的大小+ body的长度h或者最小响应的大小。
        # 否则，就设置为最小的响应大小。放回那个deferred.

    def next_response_request_deferred(self):
        # 从队列中popleft一个元组，活动请求添加request,返回这个元组，response,request,deferred
        response, request, deferred = self.queue.popleft()
        self.active.add(request)
        return response, request, deferred
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         # 从队列中popleft一个元组，活动请求添加request,返回一个元组，response,request,deferred
    def finish_response(self, response, request):
        self.active.remove(request)
        if isinstance(response, Response):
            self.active_size -= max(len(response.body), self.MIN_RESPONSE_SIZE)
        else:
            # 完成响应的话， 就从active活动列表中移除这个请求，active_size 减去对应大小。
            self.active_size -= self.MIN_RESPONSE_SIZE

    def is_idle(self):
        # 是否空闲的判断， 如果队列不为空， 或者active不为空。
        return not (self.queue or self.active)   
    def needs_backout(self):
        # 判断是否超限了。 
        return self.active_size > self.max_active_size    


class Scraper(object):
    def __init__(self, crawler):
        # 爬虫中间件管理器从crawler获取，item处理类从crawler.settings获取。然后获取一个item处理类的对象。
        # 并发item数量，信号和日志formatter设置都是从crawler获取。
        self.slot = None
        self.spidermw = SpiderMiddlewareManager.from_crawler(crawler)
        itemproc_cls = load_object(crawler.settings['ITEM_PROCESSOR'])
        self.itemproc = itemproc_cls.from_crawler(crawler)
        self.concurrent_items = crawler.settings.getint('CONCURRENT_ITEMS')
        self.crawler = crawler
        self.signals = crawler.signals
        self.logformatter = crawler.logformatter

    @defer.inlineCallbacks
    def open_spider(self, spider):
        # 这个方法就是打开给定的爬虫，并分配指定的资源，
        # 创建一个slot，然后调用对应的itemproessor类创建的处理类去打开爬虫。
        # ExecutionEngine 在open_spider里会调用这个方法来初始化scraper:
        """Open the given spider for scraping and allocate resources for it"""
        self.slot = Slot()
        yield self.itemproc.open_spider(spider)

    def close_spider(self, spider):
        """Close a spider being scraped and release its resources"""
        slot = self.slot
        slot.closing = defer.Deferred()
        slot.closing.addCallback(self.itemproc.close_spider)
        self._check_if_closing(spider, slot)
        return slot.closing  
        # 关闭爬虫并且释放资源。
        # 获取slot， 然后给slot添加一个closing的事件，然后放回方法。
    def is_idle(self):
        """Return True if there isn't any more spiders to process"""
        return not self.slot 
        # 如果没有爬虫去处理了。 就返回true了。 

    def _check_if_closing(self, spider, slot):
        if slot.closing and slot.is_idle():
            slot.closing.callback(spider)        
        # 如果closing不为空，不为空闲，  就调用指定spider的关闭回调。

    def enqueue_scrape(self, response, request, spider):
        slot = self.slot
        dfd = slot.add_response_request(response, request)
        def finish_scraping(_):
            slot.finish_response(response, request)
            self._check_if_closing(spider, slot)
            self._scrape_next(spider, slot)
            return _
        dfd.addBoth(finish_scraping)
        dfd.addErrback(
            lambda f: logger.error('Scraper bug processing %(request)s',
                                    {'request': request},
                                    exc_info=failure_to_exc_info(f),
                                    extra={'spider': spider}))
        self._scrape_next(spider, slot)
        return dfd
        # 调用add_response_request添加返回一个defferd对象，定义一个完成的方法，
        给成功和失败都添加一个finish_scraping的回调。
        # 给错误的在添加一个匿名的回调方法。
        # 调用_scrape_next 处理下一个。 
    def _scrape_next(self, spider, slot):
        while slot.queue:
            response, request, deferred = slot.next_response_request_deferred()
            self._scrape(response, request, spider).chainDeferred(deferred)   
        # 这里如果slot的queue有内容的haunted， 就一直循环下去， 调用_scrape去处理。

    def _scrape(self, response, request, spider):
        """Handle the downloaded response or failure through the spider
        callback/errback"""
        assert isinstance(response, (Response, Failure))

        dfd = self._scrape2(response, request, spider) # returns spiders processed output
        dfd.addErrback(self.handle_spider_error, request, response, spider)
        dfd.addCallback(self.handle_spider_output, request, response, spider)
        return dfd    
        # 这个方法就是处理下载响应或者失败，通过给爬虫指定的成功和错误的回调方法。
        # 先断言这个是响应流或者failure，调用_scrape2获取爬虫处理的输出
        # 添加错误回调和成功回调。
    def _scrape2(self, request_result, request, spider):
        """Handle the different cases of request's result been a Response or a
        Failure"""
        if not isinstance(request_result, Failure):
            return self.spidermw.scrape_response(
                self.call_spider, request_result, request, spider)
        else:
            # FIXME: don't ignore errors in spider middleware
            dfd = self.call_spider(request_result, request, spider)
            return dfd.addErrback(
                self._log_download_errors, request_result, request, spider)
        # 如果响应是成功的的。调用自己的爬虫中间件去处理响应。如果是错误的，调用call_spider方法，给dfd添加一个错误的回调。 

    def call_spider(self, result, request, spider):
        result.request = request
        dfd = defer_result(result)
        dfd.addCallbacks(request.callback or spider.parse, request.errback)
        return dfd.addCallback(iterate_spider_output)
        defer_result 这个方法我们上面已经看了。 主要是等100ms读写的。 添加成功的回调。
        # 这个地方注意了。 先使用request。callback , 如果没有指定的话，默认采用spider.parse方法。
        # 这就是我们的爬虫为何使用parse方法解析response的原因了。 
        # 添加一个成功回调。 iterate_spider_output 这个上面已经看过了， 就是返回一个可迭代的对象。 
    def handle_spider_error(self, _failure, request, response, spider):
        exc = _failure.value
        if isinstance(exc, CloseSpider):
            self.crawler.engine.close_spider(spider, exc.reason or 'cancelled')
            return
        logger.error(
            "Spider error processing %(request)s (referer: %(referer)s)",
            {'request': request, 'referer': referer_str(request)},
            exc_info=failure_to_exc_info(_failure),
            extra={'spider': spider}
        )
        self.signals.send_catch_log(
            signal=signals.spider_error,
            failure=_failure, response=response,
            spider=spider
        )
        self.crawler.stats.inc_value(
            "spider_exceptions/%s" % _failure.value.__class__.__name__,
            spider=spider
        )
        # 这个方法就是处理爬虫错误的 ， 如果是关闭爬虫异常， 调用对应引擎去关闭爬虫，返回
        # 其他情况，就记录下日志信息。 发送对应的爬虫错误信号， 统计信息的添加。
    def handle_spider_output(self, result, request, response, spider):
        if not result:
            return defer_succeed(None)
        it = iter_errback(result, self.handle_spider_error, request, response, spider)
        dfd = parallel(it, self.concurrent_items,
            self._process_spidermw_output, request, response, spider)
        return dfd   
        # 处理爬虫的输出，如果结果不为空 调用defer_succeed,错误的话调用错误回调，平行处理， _process_spidermw_output去处理

    def _process_spidermw_output(self, output, request, response, spider):
        """Process each Request/Item (given in the output parameter) returned
        from the given spider
        """
        if isinstance(output, Request):
            self.crawler.engine.crawl(request=output, spider=spider)
        elif isinstance(output, (BaseItem, dict)):
            self.slot.itemproc_size += 1
            dfd = self.itemproc.process_item(output, spider)
            dfd.addBoth(self._itemproc_finished, output, response, spider)
            return dfd
        elif output is None:
            pass
        else:
            typename = type(output).__name__
            logger.error('Spider must return Request, BaseItem, dict or None, '
                            'got %(typename)r in %(request)s',
                            {'request': request, 'typename': typename},
                            extra={'spider': spider})
        # 处理每个请求或者从给定的爬虫得到的item
        # 如果output是个请求的话， 调用engine.crawl抓取。 
        # 如果是baseitem或者dict的话， 处理个数加1，调用item处理类的process_item去处理item。
        # 添加处理完毕事件。 
        # 其他请求输出日志。 报告你的返回类型不是给定的item类或者字典类。 或者请求。 
        # 这里就是限定了。 我们爬虫里面的parse方法只能返回这3类的原因了。 

    def _log_download_errors(self, spider_failure, download_failure, request, spider):
        """Log and silence errors that come from the engine (typically download
        errors that got propagated thru here)
        """
        if (isinstance(download_failure, Failure) and
                not download_failure.check(IgnoreRequest)):
            if download_failure.frames:
                logger.error('Error downloading %(request)s',
                                {'request': request},
                                exc_info=failure_to_exc_info(download_failure),
                                extra={'spider': spider})
            else:
                errmsg = download_failure.getErrorMessage()
                if errmsg:
                    logger.error('Error downloading %(request)s: %(errmsg)s',
                                    {'request': request, 'errmsg': errmsg},
                                    extra={'spider': spider})

        if spider_failure is not download_failure:
            return spider_failure    
        # 这里定义个方法下载错误的， 如果是错误 并且不是ignorerequest的话进入if块。
        # 如果错误frames不为空，记录错误信息。
        # 否则调用geterrmessage方法，记录错误。
        # 如果错误不是下载错误，返回爬虫的错误。

    def _itemproc_finished(self, output, item, response, spider):
        """ItemProcessor finished for the given ``item`` and returned ``output``
        """
        self.slot.itemproc_size -= 1
        if isinstance(output, Failure):
            ex = output.value
            if isinstance(ex, DropItem):
                logkws = self.logformatter.dropped(item, ex, response, spider)
                logger.log(*logformatter_adapter(logkws), extra={'spider': spider})
                return self.signals.send_catch_log_deferred(
                    signal=signals.item_dropped, item=item, response=response,
                    spider=spider, exception=output.value)
            else:
                logger.error('Error processing %(item)s', {'item': item},
                                exc_info=failure_to_exc_info(output),
                                extra={'spider': spider})
        else:
            logkws = self.logformatter.scraped(output, response, spider)
            logger.log(*logformatter_adapter(logkws), extra={'spider': spider})
            return self.signals.send_catch_log_deferred(
                signal=signals.item_scraped, item=output, response=response,
                spider=spider)
        # itemprocess处理类结束，如果输出㐊错误。 判定他是不是dropitem。  分别记录日志。  
        # 正常情况下，记录日志 。通过日志适配器将logkws 转出logger.log方法接受的参数。 
        # 发送itemscraped信号。 

